# Librerías y configuración

In [4]:
import glob
import json
import os
import re
import time
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import nltk
import pandas as pd
import tqdm.notebook as tqdm
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel, LdaMulticore
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

os.chdir(
    "C:/Users/luism/Documents/Programacion_local/Clase/4o Año/Descubrimiento Datos Complejos/DCDC-Proyecto-Stocks/"
)
os.listdir()

BASE_PATH = "C:/Users/luism/Documents/Programacion_local/Clase/4o Año/Descubrimiento Datos Complejos/DCDC-Proyecto-Stocks/notebooks/Luis/mineria_datos"

# Cargar Datos

In [40]:
df = pd.concat(
    [pd.read_csv(file, index_col=0) for file in glob.glob("data/processed/FULL*.csv")]
)

display(df.head())
print(df.shape)

,AAL,AAME,AAON,AAPL,AAXJ,ABCB,ABIO,ACAD,ACFN,ACGL,...,XOMA,XRAY,XTLB,YNDX,YORW,ZBRA,ZEUS,ZION,ZIV,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03 00:00:00+00:00,{'val_adj_close': 4.83},{'val_adj_close': 1.83},{'val_adj_close': 5.4},{'val_adj_close': 12.4},{'val_adj_close': 43.2},{'val_adj_close': 9.73},{'val_adj_close': 9071.92},{'val_adj_close': 1.11},{'val_adj_close': 6.41},{'val_adj_close': 12.49},...,{'val_adj_close': 25.0},{'val_adj_close': 33.23},{'val_adj_close': 20.5},{'val_adj_close': 19.74},{'val_adj_close': 14.5},{'val_adj_close': 35.72},{'val_adj_close': 24.44},{'val_adj_close': 13.0},{'val_adj_close': 12.71},"{'val_adj_close': 26.46, 'noticias': [{'notici..."
2012-01-04 00:00:00+00:00,{'val_adj_close': 4.74},{'val_adj_close': 1.82},{'val_adj_close': 5.36},{'val_adj_close': 12.47},{'val_adj_close': 42.88},{'val_adj_close': 9.54},{'val_adj_close': 8981.2},{'val_adj_close': 1.09},{'val_adj_close': 6.8},{'val_adj_close': 12.27},...,{'val_adj_close': 24.8},{'val_adj_close': 33.22},{'val_adj_close': 15.5},{'val_adj_close': 19.15},{'val_adj_close': 14.26},{'val_adj_close': 35.45},{'val_adj_close': 23.4},"{'val_adj_close': 13.29, 'noticias': [{'notici...",{'val_adj_close': 12.93},"{'val_adj_close': 26.33, 'noticias': [{'notici..."
2012-01-05 00:00:00+00:00,{'val_adj_close': 5.16},{'val_adj_close': 1.88},{'val_adj_close': 5.33},{'val_adj_close': 12.61},{'val_adj_close': 42.88},{'val_adj_close': 9.59},{'val_adj_close': 8891.38},{'val_adj_close': 1.1},{'val_adj_close': 6.62},{'val_adj_close': 12.5},...,"{'val_adj_close': 25.0, 'noticias': [{'noticia...",{'val_adj_close': 33.24},{'val_adj_close': 17.5},{'val_adj_close': 19.29},{'val_adj_close': 14.09},{'val_adj_close': 35.4},{'val_adj_close': 23.33},{'val_adj_close': 13.66},{'val_adj_close': 13.03},"{'val_adj_close': 30.95, 'noticias': [{'notici..."
2012-01-06 00:00:00+00:00,{'val_adj_close': 5.28},{'val_adj_close': 1.84},{'val_adj_close': 5.27},{'val_adj_close': 12.74},{'val_adj_close': 42.5},{'val_adj_close': 9.79},{'val_adj_close': 9253.35},{'val_adj_close': 1.15},{'val_adj_close': 6.67},{'val_adj_close': 12.49},...,{'val_adj_close': 25.6},{'val_adj_close': 33.05},{'val_adj_close': 17.5},{'val_adj_close': 18.61},{'val_adj_close': 14.07},{'val_adj_close': 35.11},{'val_adj_close': 22.78},"{'val_adj_close': 13.68, 'noticias': [{'notici...",{'val_adj_close': 13.18},"{'val_adj_close': 31.27, 'noticias': [{'notici..."
2012-01-09 00:00:00+00:00,{'val_adj_close': 5.39},{'val_adj_close': 1.87},"{'val_adj_close': 5.26, 'noticias': [{'noticia...",{'val_adj_close': 12.72},{'val_adj_close': 42.67},{'val_adj_close': 9.78},{'val_adj_close': 9163.54},{'val_adj_close': 1.19},{'val_adj_close': 6.67},{'val_adj_close': 12.4},...,{'val_adj_close': 27.8},{'val_adj_close': 32.89},{'val_adj_close': 18.5},{'val_adj_close': 18.73},{'val_adj_close': 13.88},{'val_adj_close': 34.95},{'val_adj_close': 22.48},{'val_adj_close': 13.87},{'val_adj_close': 13.23},{'val_adj_close': 31.78}


(2260, 1149)


# 1. Transformar celdas en True o [lista_de_noticias]

In [44]:
def parse(x):
    pbar.update(1)
    noticias = eval(x).get("noticias")
    if noticias:
        return [noticia["noticia"] for noticia in noticias]
    return None


pbar = tqdm.tqdm(total=df.shape[0] * df.shape[1])
df_noticias = df.map(parse)
pbar.close()

display(df_noticias.head())
display(df_noticias.shape)

  0%|          | 0/2596740 [00:00<?, ?it/s]

,AAL,AAME,AAON,AAPL,AAXJ,ABCB,ABIO,ACAD,ACFN,ACGL,...,XOMA,XRAY,XTLB,YNDX,YORW,ZBRA,ZEUS,ZION,ZIV,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03 00:00:00+00:00,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"[Benzinga's Top Downgrades, UPDATE: Piper Jaff..."
2012-01-04 00:00:00+00:00,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,"[First Niagara: Financial Winner, First Niagar...",None,"[Zumiez Jumps 11% on December Sales Results, G..."
2012-01-05 00:00:00+00:00,None,None,None,None,None,None,None,None,None,None,...,"[XOMA Corporation Halted for Pending News, XOM...",None,None,None,None,None,None,None,None,[UPDATE: Oppenheimer Raises Price Target on Zu...
2012-01-06 00:00:00+00:00,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,[7 Surprise Mid-Cap Bank Picks for the Fourth ...,None,"[Stock Futures Cut Losses on Jobs Data, Stocks..."
2012-01-09 00:00:00+00:00,None,None,[Top 4 Stocks In The General Building Material...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


(2260, 1149)

In [ ]:
df_noticias.to_pickle(f"{BASE_PATH}/Datos/01_df_contiene_noticia.pkl")

# 2. Guardar cada noticia con su fecha

In [53]:
news = []
dates = []

p_bar = tqdm.tqdm(total=df_noticias.shape[0] * (df_noticias.shape[1] - 1))

for i in range(df_noticias.shape[0]):
    for j in range(1, df_noticias.shape[1]):
        if df_noticias.iloc[i, j]:
            date = df_noticias.index[i]
            len_news = len(df_noticias.iloc[i, j])
            dates_to_add = [date] * len_news

            news.extend(df_noticias.iloc[i, j])
            dates.extend(dates_to_add)
        p_bar.update(1)

p_bar.close()

  0%|          | 0/2594480 [00:00<?, ?it/s]

In [54]:
df_noticias_fecha = pd.DataFrame({"date": dates, "news": news})
display(df_noticias_fecha.head())
display(df_noticias_fecha.shape)

,date,news
0,2012-01-03 00:00:00+00:00,'Mad Money Lightning Round': Stay Long Sandridge
1,2012-01-03 00:00:00+00:00,Cramer's 'Mad Money' Recap: Betting Against th...
2,2012-01-03 00:00:00+00:00,'Mad Money Lightning Round': Stay Long Sandridge
3,2012-01-03 00:00:00+00:00,BioLineRx Announces Initiation of CE Mark Regi...
4,2012-01-03 00:00:00+00:00,Hearing Rumors of the Death of Fidel Castro


(2303734, 2)

In [ ]:
df_noticias_fecha.to_pickle(f"{BASE_PATH}/Datos/02_df_noticias_fecha.pkl")

# 3. Procesar textos y añadir mes y año

In [6]:
# Preprocesamiento de noticias
def preprocess_text(text, p_bar):
    if not text:
        p_bar.update(1)
        return []
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    tokens = word_tokenize(text)
    tokens = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token not in stop_words and len(token) > 2
    ]
    p_bar.update(1)
    return tokens

In [8]:
# Barra de progreso para el preprocesamiento de noticias
p_bar = tqdm.tqdm(total=df_noticias_fecha.shape[0], desc="Preprocessing news")

# Aplica la función de preprocesamiento a la columna 'news'
df_noticias_fecha["news_processed"] = df_noticias_fecha["news"].apply(
    preprocess_text, args=(p_bar,)
)
p_bar.close()

Preprocessing news:   0%|          | 0/2303734 [00:00<?, ?it/s]

In [19]:
df_noticias_fecha["date"] = pd.to_datetime(df_noticias_fecha["date"])
df_noticias_fecha["month"] = df_noticias_fecha["date"].dt.to_period("M")
df_noticias_fecha = df_noticias_fecha.loc[
    :, ["date", "month", "news", "news_processed"]
]

C:\Users\luism\AppData\Local\Temp\ipykernel_50216\3953517154.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_noticias_fecha["month"] = df_noticias_fecha["date"].dt.to_period("M")


In [24]:
display(df_noticias_fecha.head())
display(df_noticias_fecha.shape)

,date,month,news,news_processed
0,2012-01-03 00:00:00+00:00,2012-01,'Mad Money Lightning Round': Stay Long Sandridge,"[mad, money, lightning, round, stay, long, san..."
1,2012-01-03 00:00:00+00:00,2012-01,Cramer's 'Mad Money' Recap: Betting Against th...,"[cramers, mad, money, recap, betting, bear, fi..."
2,2012-01-03 00:00:00+00:00,2012-01,'Mad Money Lightning Round': Stay Long Sandridge,"[mad, money, lightning, round, stay, long, san..."
3,2012-01-03 00:00:00+00:00,2012-01,BioLineRx Announces Initiation of CE Mark Regi...,"[biolinerx, announces, initiation, mark, regis..."
4,2012-01-03 00:00:00+00:00,2012-01,Hearing Rumors of the Death of Fidel Castro,"[hearing, rumor, death, fidel, castro]"


(2303734, 4)

In [21]:
df_noticias_fecha.to_pickle(f"{BASE_PATH}/Datos/03_df_noticias_fecha_procesado.pkl")

Con esto tenemos un df ordenado como punto de partida para luego coger un sample y preprocesarlo para el modelo LDA o DTM.